# Predicción de Accidentes Cerebrovasculares Utilizando Técnicas de Aprendizaje Automático

## Análisis Exploratorio de Datos

## Preprocesamiento de Datos e Ingeniería de Características

## Desarrollo y Evaluación de Modelos

## Ajuste de Hiperparámetros y Selección de Modelos

## Técnicas Avanzadas